In [1]:
import sys
sys.path.append('..')
import configs.common as cc
import configs.mamba as cm
import models
import processing
import torch.optim as optim
import torch.nn as nn
import torch

In [8]:
train_dataloader, test_dataloader = processing.get_train_test_dataloaders('F:\\GitHub\\dataset\\np_dataset')
METADATA_VOCAB_SIZE = processing.get_metadata_vocab_size()

In [6]:
mv = cm.config.model_values
args = models.ModelArgs(
    d_model = mv.d_model,
    n_layer = mv.n_layer,
    vocab_size = cc.vocab_size,
    d_state = mv.d_state,
    expand = mv.expand,
    dt_rank = mv.dt_rank,
    d_conv = mv.d_conv ,
    pad_vocab_size_multiple = mv.pad_vocab_size_multiple,
    conv_bias = mv.conv_bias,
    bias = mv.bias
)

model = models.Mamba(args)
model.to(cc.config.values.device)

Mamba(
  (embedding): Embedding(1488, 12)
  (layers): ModuleList(
    (0-11): 12 x ResidualBlock(
      (mixer): MambaBlock(
        (in_proj): Linear(in_features=12, out_features=96, bias=False)
        (conv1d): Conv1d(48, 48, kernel_size=(6,), stride=(1,), padding=(5,), groups=48)
        (x_proj): Linear(in_features=48, out_features=128, bias=False)
        (dt_proj): Linear(in_features=64, out_features=48, bias=True)
        (out_proj): Linear(in_features=48, out_features=12, bias=False)
      )
      (norm): RMSNorm()
    )
  )
  (norm_f): RMSNorm()
  (lm_head): Linear(in_features=12, out_features=1488, bias=False)
)

In [9]:
for batch_idx, (src, trg, metadata) in enumerate(train_dataloader):
        # Forward pass
        1 + 1
output = model(src)
output.argmax(-2)

tensor([[ 60,  13,  22,  ...,  24,  43,  78],
        [ 34,  96, 106,  ...,  25, 115,  78],
        [ 98,  60,  24,  ...,  50, 120, 122]], device='cuda:0')

In [44]:

# vocab_size, n_embd, n_layer, n_heads, block_size, dropout, device
# model = models.Transformer(VOCAB_SIZE, N_EMBD, N_LAYER, N_HEAD, BLOCK_SIZE, DROPOUT, DEVICE)
model.to(DEVICE)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

# Training loop
num_epochs = EPOCHS
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    total_loss = 0

    for batch_idx, (src, trg, metadata) in enumerate(train_dataloader):
        # Forward pass
        output = model(src)
        # print(output.shape)
        # Reshape output and target for loss calculation
        output = output.reshape(-1, VOCAB_SIZE)  # Flatten the output to [batch_size * seq_len, vocab_size]
        trg = trg.view(-1)  # Flatten the target to [batch_size * seq_len]

        # Compute loss
        loss = criterion(output, trg)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()

        if (batch_idx + 1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{batch_idx+1}/{len(train_dataloader)}], Loss: {loss.item():.4f}')

    avg_loss = total_loss / len(train_dataloader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Average Loss: {avg_loss:.4f}')

    # Validation loop (optional)
    model.eval()  # Set the model to evaluation mode
    val_loss = 0
    with torch.no_grad():
        for src, trg, metadata in test_dataloader:
            src, trg = src.to(DEVICE), trg.to(DEVICE)
            output = model(src)
            output = output.reshape(-1, VOCAB_SIZE)
            trg = trg.view(-1)
            val_loss += criterion(output, trg).item()
    
    avg_val_loss = val_loss / len(test_dataloader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Validation Loss: {avg_val_loss:.4f}')

print("Training complete!")

Epoch [1/200], Step [10/14], Loss: 2.9573
Epoch [1/200], Average Loss: 3.2163
Epoch [1/200], Validation Loss: 3.3036
Epoch [2/200], Step [10/14], Loss: 2.8287
Epoch [2/200], Average Loss: 3.0894
Epoch [2/200], Validation Loss: 3.0650
Epoch [3/200], Step [10/14], Loss: 3.2510
Epoch [3/200], Average Loss: 3.1139
Epoch [3/200], Validation Loss: 3.6133
Epoch [4/200], Step [10/14], Loss: 3.1651
Epoch [4/200], Average Loss: 3.1490
Epoch [4/200], Validation Loss: 3.3505
Epoch [5/200], Step [10/14], Loss: 3.0291
Epoch [5/200], Average Loss: 3.0178
Epoch [5/200], Validation Loss: 3.7025
Epoch [6/200], Step [10/14], Loss: 3.0166
Epoch [6/200], Average Loss: 3.0707
Epoch [6/200], Validation Loss: 3.5024
Epoch [7/200], Step [10/14], Loss: 3.1116
Epoch [7/200], Average Loss: 3.0511
Epoch [7/200], Validation Loss: 2.9242


KeyboardInterrupt: 

In [1]:
print(output.shape)
print(output.argmax(-2)[:40])
print(trg.view(-1)[:40])

NameError: name 'output' is not defined